# libraries

In [21]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm 
from datetime import datetime

# First Part (create the data base)

In [22]:
def CreateDataBase(database, user, password, host, port, newBase):

    sqlQuery = 'CREATE database ' + newBase
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))
    conn = engine.connect()
    conn.execute("commit")

    conn.execute(sqlQuery)
    print("Database " + newBase +" created successfully........")
    conn.close()


    engine.dispose()


In [23]:
CreateDataBase(database="postgres", user='postgres', password='20906540', host='127.0.0.1', port= '5432', newBase = "factory" )

Database factory created successfully........


# Second Part (Creat a table from a DataFrame)

In [24]:
def createTable(data, table, database, user, password, host, port):
    

    data.location = data.location.apply(lambda x: str(x).replace("'", "’"))
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))

    k = []
    for n in range(len(data.dtypes)):
        if data.dtypes[n] == 'object':
            k.append(data.dtypes.index[n] + " TEXT")
        if data.dtypes[n] == 'int64' or data.dtypes[n] == 'float64':
            k.append(data.dtypes.index[n] + " FLOAT")


    k1 = list(data.dtypes.index)

    sql = 'CREATE TABLE '+table+'(' + ", ".join(k) + ", UNIQUE (" + ", ".join(k1)+"))" 
    data = data.fillna("NoValue")
    data = data.set_index(data.columns[0])
    engine.execute(sql)
    
    data = data.reset_index()



    for m in tqdm(range(len(data))):
        k1 = list(data.loc[m].fillna("NoValue").index)
        k1 = [str(a) for a in k1]
        k1 = ", ".join(k1)


        k = data.loc[m].fillna("NoValue").to_list()
        for i in range(len(k)):
            if str(type(k[i])) == "<class 'str'>":
                k[i] = "'" + k[i] + "'"
        k = [str(a) for a in k]
        k = [a.replace(",", "") for a in k]
        k = ", ".join(k)

        try:
            sql = '''INSERT INTO public.'''+ table +'''('''+k1+ ") VALUES ("+k+ ") ON CONFLICT DO NOTHING"
            engine.execute(sql)
        except:
            print("Fila " + str(m) +" no insertada")


In [25]:
data = pd.read_csv("table_1.csv")
data.head()

,id,author,subreddit,date,text,source,location,negative,neutral,positive,compound,analysis,keyword
0,6c51302b-2a08-4010-9d9b-88977791a1ea,pennykirk583,NaN,1627572077,is that joe biden,twitter,"Seattle, WA",0.000,1.000,0.000,0.0000,neutral,Biden
1,6c51302b-2a08-4010-9d9b-88977791a1ea,mdbsat,NaN,1627572077,i naively thought once biden won i would stop ...,twitter,"San Antonio, TX",0.292,0.417,0.292,0.0000,neutral,Biden
2,6c51302b-2a08-4010-9d9b-88977791a1ea,davidbr11505741,NaN,1627572077,postal union refuses bidens mandatory vaccinat...,twitter,"Georgia, USA",0.000,0.874,0.126,0.0772,positive,Biden
3,6c51302b-2a08-4010-9d9b-88977791a1ea,ENDOFET3RNITY,NaN,1627572076,joe biden has unfollowed dababy on instagram,twitter,NaN,0.000,1.000,0.000,0.0000,neutral,Biden
4,6c51302b-2a08-4010-9d9b-88977791a1ea,yanetCornell,NaN,1627572076,barr comes back to help trump dismantle bidenp...,twitter,NaN,0.189,0.651,0.160,-0.1280,negative,Biden


In [26]:
createTable(data = data, table = "tabla", database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')

 76%|██████████████████████████████████████████████████████████▊                  | 1646/2154 [00:04<00:01, 390.19it/s]

Fila 1580 no insertada


100%|█████████████████████████████████████████████████████████████████████████████| 2154/2154 [00:05<00:00, 389.16it/s]


# Third Part (Crear View)

In [27]:
def Views(database, user, password, host, port, sqlQuery):
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))


    sql = sqlQuery
    try:
        engine.execute(sql)
    except:
        print("Fila " + str(n) +" no insertada")
            
    engine.dispose()

In [28]:
sqlQuery ='''CREATE VIEW Vista1 AS
SELECT  keyword, id , date,
AVG(positive) as avg_positive,
AVG(neutral) as avg_neutral,
AVG(negative) as avg_negative
FROM    tabla
GROUP BY 
keyword, id , date
'''

Views(database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432', sqlQuery = sqlQuery )

In [29]:
sqlQuery ='''CREATE VIEW Vista2 AS
SELECT  date,keyword, 
sum(CASE WHEN positive>negative AND positive>neutral THEN 1 ELSE 0 END) AS count_positive,
sum(CASE WHEN negative>positive AND negative>neutral THEN 1 ELSE 0 END) AS count_negative,
sum(CASE WHEN neutral>positive AND neutral>negative THEN 1 ELSE 0 END) AS count_neutral
FROM  tabla
GROUP BY
date,keyword
'''

Views(database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432', sqlQuery = sqlQuery )


# Four Part (Insert without Duplicated)

In [30]:
def InsertInTable(data, table, database, user, password, host, port):
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))

    data = pd.read_csv("table_1.csv")
    data.location = data.location.apply(lambda x: str(x).replace("'", "’"))
    data.text = data.location.apply(lambda x: str(x).replace(",", " "))

    for n in tqdm(range(len(data))):
        k1 = list(data.loc[n].fillna("NoValue").index)
        k1 = [str(a) for a in k1]
        k1 = ", ".join(k1)


        k = data.loc[n].fillna("NoValue").to_list()
        for i in range(len(k)):
            if str(type(k[i])) == "<class 'str'>":
                k[i] = "'" + k[i] + "'"
        k = [str(a) for a in k]
        k = [a.replace(",", "") for a in k]
        k = ", ".join(k)


        sql = '''INSERT INTO public.'''+ table +'''('''+k1+ ") VALUES ("+k+ ") ON CONFLICT DO NOTHING"
        try:
            engine.execute(sql)
        except:
            print("Fila " + str(n) +" no insertada")
            
    engine.dispose()

In [31]:
data = pd.read_csv("table_2.csv")
data.head()

,id,author,subreddit,date,text,source,location,negative,neutral,positive,compound,analysis,keyword
0,6c51302b-2a08-4010-9d9b-88977791a1ea,pennykirk583,NaN,1627572077,is that joe biden,twitter,"Seattle, WA",0.000,1.000,0.000,0.0000,neutral,Biden
1,6c51302b-2a08-4010-9d9b-88977791a1ea,mdbsat,NaN,1627572077,i naively thought once biden won i would stop ...,twitter,"San Antonio, TX",0.292,0.417,0.292,0.0000,neutral,Biden
2,6c51302b-2a08-4010-9d9b-88977791a1ea,davidbr11505741,NaN,1627572077,postal union refuses bidens mandatory vaccinat...,twitter,"Georgia, USA",0.000,0.874,0.126,0.0772,positive,Biden
3,6c51302b-2a08-4010-9d9b-88977791a1ea,ENDOFET3RNITY,NaN,1627572076,joe biden has unfollowed dababy on instagram,twitter,NaN,0.000,1.000,0.000,0.0000,neutral,Biden
4,6c51302b-2a08-4010-9d9b-88977791a1ea,yanetCornell,NaN,1627572076,barr comes back to help trump dismantle bidenp...,twitter,NaN,0.189,0.651,0.160,-0.1280,negative,Biden


In [32]:
InsertInTable(data,"tabla", database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')

100%|█████████████████████████████████████████████████████████████████████████████| 2154/2154 [00:05<00:00, 391.98it/s]


# fifth part (extract information from views)

In [33]:
def ReadView(sqlQuery, database, user, password, host, port):
    
    engine = create_engine(str('postgresql://postgres:'+password+'@'+host+':'+port+'/'+database))

    data = pd.read_sql_query(sqlQuery,con=engine)
    engine.dispose()
    return(data)

In [34]:
query = "SELECT * FROM public.vista1"
view1 = ReadView(sqlQuery = query, database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')
view1.head()

,keyword,id,date,avg_positive,avg_neutral,avg_negative
0,Biden,c913dbcc-cda8-4039-b355-ee3334e9b9e5,1.627673e+09,0.000000,1.000000,0.000
1,Biden,ed39a761-f819-44c6-9eba-b3e1f0be7480,1.627673e+09,0.042000,0.708000,0.250
2,Biden,04bc808b-5d54-4208-9909-60c547d4805e,1.627653e+09,0.000000,1.000000,0.000
3,Biden,ac5fd83e-3b22-4f68-b96b-b0589ba50be7,1.627673e+09,0.192333,0.674667,0.133
4,Biden,ca69e9a9-55ba-4f77-867a-ef80c585d9ec,1.627572e+09,0.055000,0.697000,0.249


In [35]:
query = "SELECT * FROM public.vista2"
view2 = ReadView(sqlQuery = query, database="factory", user='postgres', password='20906540', host='127.0.0.1', port= '5432')
view2.head()

,date,keyword,count_positive,count_negative,count_neutral
0,1.627573e+09,Biden,0,0,2
1,1.627653e+09,Biden,0,0,4
2,1.627644e+09,Biden,0,0,2
3,1.627597e+09,Biden,0,0,8
4,1.627572e+09,Biden,0,0,2


# Sixth part (graphic)

In [37]:
view1.date = view1.date.apply(lambda x: datetime.fromtimestamp(x))
view2.date = view2.date.apply(lambda x: datetime.fromtimestamp(x))

In [38]:
data = view1[view1["date"]>="2021-07-28"][["date", 'avg_positive', "avg_neutral", "avg_negative"]]
data = data.resample('60min', on='date').mean().reset_index()

In [39]:
data.head()

,date,avg_positive,avg_neutral,avg_negative
0,2021-07-28 23:00:00,0.024636,0.783318,0.192091
1,2021-07-29 00:00:00,0.070417,0.692583,0.112083
2,2021-07-29 01:00:00,0.028750,0.815167,0.155917
3,2021-07-29 02:00:00,0.000000,0.923000,0.077000
4,2021-07-29 03:00:00,0.029091,0.851591,0.119409


In [40]:
import plotly.express as px
fig = px.bar(data, x='date', y=["avg_positive", "avg_neutral","avg_negative"],color_discrete_sequence = ["blue","gray","red"])
fig.show()

# Count with neutral

In [49]:
data = view2[view2["date"]>="2021-07-28"][["date", 'count_positive', "count_neutral", "count_negative"]]
#data = data.resample('60min', on='date').count().reset_index()
#data['date'] = pd.to_datetime(df1['TIME'])
df = data.resample('60min', on='date').sum()
df

,count_positive,count_neutral,count_negative
date,,,
2021-07-28 23:00:00,0,24,2
2021-07-29 00:00:00,0,20,2
2021-07-29 01:00:00,0,16,0
2021-07-29 02:00:00,0,6,0
2021-07-29 03:00:00,0,28,0
2021-07-29 04:00:00,2,14,0
2021-07-29 05:00:00,0,6,0
2021-07-29 06:00:00,0,26,0
2021-07-29 07:00:00,2,20,0


In [52]:
import plotly.express as px
fig = px.line(df)
fig.show()

# Count without neutral

In [54]:
df = data.resample('60min', on='date').sum()
df1 = df.drop("count_neutral", axis = 1)

In [56]:
fig = px.line(df1)
fig.show()